# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
weather_file = pd.read_csv("../output_files/city_weatherdata.csv")
weather_file

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Zabul Province,32.2500,67.2500,36.01,62,100,5.12,AF,1643499952
1,1,Ushuaia,-54.8000,-68.3000,44.26,87,100,13.80,AR,1643499462
2,2,New Norfolk,-42.7826,147.0587,73.65,33,13,2.01,AU,1643499466
3,3,Tiksi,71.6872,128.8694,-9.49,98,56,4.25,RU,1643499261
4,4,Arlit,18.7369,7.3853,61.97,16,1,6.67,NE,1643499955
...,...,...,...,...,...,...,...,...,...,...
559,559,Manyana,-23.4000,21.7167,71.89,72,84,14.63,BW,1643500870
560,560,Ngunguru,-35.6167,174.5000,72.16,64,40,11.16,NZ,1643500871
561,561,Lumby,50.2498,-118.9690,32.04,95,100,1.10,CA,1643500931
562,562,Anloga,5.7947,0.8973,79.59,84,90,4.85,GH,1643500932


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
#configure gmaps
gmaps.configure(api_key=g_key)

#store lat and lng as locations and humidity as weight
locations = weather_file[["Lat","Lng"]]
humidity = weather_file["Humidity"].astype(float)

In [21]:
#plot heatmap
fig = gmaps.figure(center = [0,0], zoom_level=2)

#create and add heat layer
heat_layer = gmaps.heatmap_layer(locations,
                                 weights = humidity,
                                 dissipating = False,
                                 max_intensity = np.max(humidity),
                                 point_radius = 2)
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit ideal weather conditions.
* Drop any rows with null values.

In [44]:
idealweather_df = weather_file.loc[(weather_file["Wind Speed"] <= 10) 
                                   & (weather_file["Cloudiness"] == 0) 
                                   & (weather_file["Max Temp"] >= 60)
                                   & (weather_file["Max Temp"] <= 80)].dropna().drop("City ID", 1).reset_index(drop=True)
idealweather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Geraldton,-28.7667,114.6000,75.61,73,0,8.05,AU,1643499976
1,Illapel,-31.6308,-71.1653,65.77,72,0,5.50,CL,1643499977
2,Ojinaga,29.5667,-104.4167,63.57,41,0,0.00,MX,1643499493
3,Pacific Grove,36.6177,-121.9166,69.48,54,0,1.01,US,1643500058
4,Cape Town,-33.9258,18.4232,68.41,70,0,7.00,ZA,1643499434
5,Cayenne,4.9333,-52.3333,79.09,86,0,0.00,GF,1643500127
6,Mar del Plata,-38.0023,-57.5575,78.82,47,0,6.91,AR,1643500140
7,Esperance,-33.8667,121.9000,69.64,59,0,9.89,AU,1643500214
8,Uvalde,29.3669,-99.7670,63.55,20,0,6.91,US,1643500216
9,Morondava,-20.2833,44.2833,78.31,83,0,6.87,MG,1643500293


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
#create hotel dataframe, narrowed to certain columns
hotel_df = idealweather_df.loc[:,["City","Country","Lat","Lng"]]

#add "Hotel Name" column
hotel_df["Hotel Name"] = ""

#show new df
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Geraldton,AU,-28.7667,114.6000,
1,Illapel,CL,-31.6308,-71.1653,
2,Ojinaga,MX,29.5667,-104.4167,
3,Pacific Grove,US,36.6177,-121.9166,
4,Cape Town,ZA,-33.9258,18.4232,
5,Cayenne,GF,4.9333,-52.3333,
6,Mar del Plata,AR,-38.0023,-57.5575,
7,Esperance,AU,-33.8667,121.9000,
8,Uvalde,US,29.3669,-99.7670,
9,Morondava,MG,-20.2833,44.2833,


In [58]:
#set parameters to search for hotels within 5000 meters
params = {"radius": 5000,
          "type": "hotel",
          "keyword": "hotels",
          "key": g_key}

#base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#use lat/lng to find hotels
for index, row in hotel_df.iterrows():
    
    #get lat, lng, city name
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    #change location each iteration and leave params 
    params["location"] = f"{lat},{lng}"
    
    #api request, converted to json; results saved as variable for cleanliness later
    hotel_query = requests.get(base_url, params=params).json()
    results = hotel_query["results"]
    
    #print statement to pretty-up the output
    print(f"Retrieving results for city {index + 1}: {city_name}")
    
    #save the hotel names into a dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
    #if no hotel available
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("--------------------------------")
    
    #adding 1 sec pause to avoid SSL error

#print to denote end of search
print("-------End of Search-------")

Retrieving results for city 1: Geraldton
Closest hotel in Geraldton is Ocean Centre Hotel
--------------------------------
Retrieving results for city 2: Illapel
Closest hotel in Illapel is Hotel Casablanca
--------------------------------
Retrieving results for city 3: Ojinaga
Closest hotel in Ojinaga is Riata Inn Presidio
--------------------------------
Retrieving results for city 4: Pacific Grove
Closest hotel in Pacific Grove is The Monterey Hotel
--------------------------------
Retrieving results for city 5: Cape Town
Closest hotel in Cape Town is The Table Bay hotel
--------------------------------
Retrieving results for city 6: Cayenne
Closest hotel in Cayenne is Mercure Cayenne Royal Amazonia
--------------------------------
Retrieving results for city 7: Mar del Plata
Closest hotel in Mar del Plata is Sheraton Mar del Plata Hotel
--------------------------------
Retrieving results for city 8: Esperance
Closest hotel in Esperance is Comfort Inn Bay Of Isles
------------------

In [63]:
#display hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Geraldton,AU,-28.7667,114.6000,Ocean Centre Hotel
1,Illapel,CL,-31.6308,-71.1653,Hotel Casablanca
2,Ojinaga,MX,29.5667,-104.4167,Riata Inn Presidio
3,Pacific Grove,US,36.6177,-121.9166,The Monterey Hotel
4,Cape Town,ZA,-33.9258,18.4232,The Table Bay hotel
5,Cayenne,GF,4.9333,-52.3333,Mercure Cayenne Royal Amazonia
6,Mar del Plata,AR,-38.0023,-57.5575,Sheraton Mar del Plata Hotel
7,Esperance,AU,-33.8667,121.9000,Comfort Inn Bay Of Isles
8,Uvalde,US,29.3669,-99.7670,
9,Morondava,MG,-20.2833,44.2833,Palissandre Cote Ouest resort & SPA


In [67]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [70]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))